## 0. Set up

In [11]:
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## 1. Bases de dados

In [12]:
covid19_br_kaggel = pd.read_csv("../data/coronavirus_brazil_kaggle/brazil_covid19.csv")
covid19_br_vicosa_cites = pd.read_csv("../data/coronavirus_brazil_vicosa/cases-brazil-cities.csv")
covid19_br_vicosa_cites_times = pd.read_csv("../data/coronavirus_brazil_vicosa/cases-brazil-cities-time.csv")
covid19_br_vicosa_states = pd.read_csv("../data/coronavirus_brazil_vicosa/cases-brazil-states.csv")
covid19_br_vicosa_brazil = pd.read_csv("../data/coronavirus_brazil_vicosa/cases-brazil-total.csv")
gov_br_novocoronavirus = pd.read_csv("../data/coronavirus_brazil_vicosa/plataforma-saude-gov-br-novocoronavirus.csv")

#### 1.1 Exploração de dados premliminar

In [13]:
covid19_br_kaggel.head()

,date,hour,state,suspects,refuses,cases,deaths
0,2020-01-30,16:00,Minas Gerais,1,0,0,0
1,2020-01-30,16:00,Rio de Janeiro,1,0,0,0
2,2020-01-30,16:00,Santa Catarina,0,2,0,0
3,2020-01-30,16:00,São Paulo,3,1,0,0
4,2020-01-30,16:00,Rio Grande do Sul,2,2,0,0


In [15]:
covid19_br_kaggel.shape

(900, 7)

In [18]:
print("Data mínima: {0}\n".format(min(covid19_br_kaggel['date'])))

Data mínima: 2020-01-30

